## Feature Selection: Variáveis constantes e quase-constantes
Neste artigo, você encontra uma breve explicação sobre feature selection e um tutorial de como lidar com variáveis constantes e quase-constantes. 

Um conjunto de dados geralmente começa com um grande número de features (variáveis), porém podemos empregar uma variedade de métodos com o objetivo de determinar quais dessas features são realmente importantes para realizar as previsões. Esse procedimento é conhecido como feature selection ou seleção de variáveis.

Feature selection pode ser definido como o processo na qual você seleciona automaticamente ou manualmente aquelas features que mais contribuem para a predição da variável target (variável de previsão). Em outras palavras, é o processo de selecionar um subconjunto de features relevantes do seu conjunto total de dados, com o objetivo de construir um algoritmo de machine learning. As features que são utilizadas para treinar seu modelo de machine learning, posuem uma grande influência no resultado que seu modelo pode alcançar.



Quais os benefícios de aplicar feature selection? Abaixo indico alguns benefícios, porém existem outros.
- Redução do overfitting 
- Melhora da acurácia
- Redução do tempo de treinamento do modelo
- Modelos com menor propensão a erros

Vários métodos foram desenvolvidos para selecionar aquelas features ideais para a construção de um algoritmo de machine learning. As técnicas utilizadas podem ser divididas em três categorias principais:
1. **Filter Methods**: Métodos de seleção que utilizam medidas estatísticas para atribuir um score para cada feature. As features são classificadas pelo score para serem mantidas ou removidas do modelo. Normalmente se usam testes univariados que consideram a independência da feature com a variável alvo.
2. **Wrapper Methods** : Métodos de seleção que selecionam um conjunto de features, onde diferentes combinações são preparadas, avaliadas e comparadas. Um modelo preditivo é usado para avaliar a combinação de features a atribuir um score baseado em uma acurácia de modelo. 
3. **Embedded Methods**: Métodos Embedded aprendem quais features melhor contribuiem para a acurácia do modelo no momento de construção do modelo.

Neste artigo, estarei concentrado nos filter methods, mais precisamente em um método básico e simples, porém bastante eficiente, utilizado para selecionar variáveis constantes e quase-constantes. Esse método baseia-se apenas na variância de cada feature. Sendo as features com variância igual a 0, consideradas como constantes, uma vez que seus valores são o mesmo para todos os registros. Já as features quase-constantes, possuem uma variância muito pequena, ou seja, seu valor é quase sempre o mesmo em todo o dataset, por isso são chamadas de quase-constantes.

Quando trabalhamos com um conjunto de dados muito extenso, com muitas features, pode ser complicado e até mesmo inviável determinar quais features possuem valores constantes ou quase-constantes. Dessa forma, é importante que o trabalho seja automatizado, fazendo com que todo o processo ocorra de maneira mais rápida, além de manter a qualidade da análise. Para que isso seja possível, podemos utilizar o método VarianceThreshold pertencente a biblioteca sklearn,da linguagem de programação Python. O método é bastante simples e consiste em atribuir um threshold, ou seja, um limiar, um valor na qual servirá como um filtro, removendo as features que apresentarem limite inferior a variância informada.

Todo o processo será feito através da linguagem Python e o conjunto de dados utilizado pode ser encontrado através do link https://www.kaggle.com/c/santander-customer-satisfaction. Esse dataset, refere-se a dados bancários de clientes do banco Santander, sendo as features anônimas. Sem mais delongas, vamos para a parte prática.

Para dar início ao trabalho, vamos importar as três bibliotecas a serem utilizadas e carregar os dados de treino e de teste.

In [1]:
# Importando as bibliotecas
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold

# Dados de treino e teste
treino = pd.read_csv('C:/Users/otavio/Mundo/medium/data/santander-train.csv', index_col = 0)
teste  = pd.read_csv('C:/Users/otavio/Mundo/medium/data/santander-test.csv', index_col = 0)

Vamos remover a coluna target dos dados de treino e analisar o formato dos dados.

In [2]:
# Removendo coluna TARGET dos dados de treino
treino.drop(labels = ['TARGET'], axis = 1, inplace = True)

In [3]:
# Shape dos dados de treino e teste
print('Dados de treino:', treino.shape)
print('Dados de teste :', teste.shape)

Dados de treino: (76020, 369)
Dados de teste : (75818, 369)


Através do código acima, podemos observar que os datasets apresentam 369 features. 

Primeiramente, vamos analisar se o conjunto de dados apresenta **features constantes**, além de determinar quantas e quais são essas features. Para isso, será aplicado o método VarianceThreshold, aplicando um threshold igual a 0, ou seja, iremos mapear aquelas features que possuem variância igual a zero através do fit.

In [4]:
vt = VarianceThreshold(threshold = 0)
vt.fit(treino)

VarianceThreshold(threshold=0)

O método get_support é um vetor booleano que indica quais features são retidas. Se somarmos get_support, obtemos quantas features NÃO são constantes. De todas as 369 features, 335 apresentaram variância diferente de 0.

In [5]:
sum(vt.get_support())

335

Com poucas linhas de código, foi possível determinar que 34 features apresentaram variância igual a 0 e criar uma lista de quais as features que serão excluídas dos datasets de treino e teste. 

In [6]:
# Features constantes
features_constantes = [feature for feature in treino.columns if feature not in treino.columns[vt.get_support()]]
print('Número de features constantes: ', len(features_constantes))
features_constantes

Número de features constantes:  34


['ind_var2_0',
 'ind_var2',
 'ind_var27_0',
 'ind_var28_0',
 'ind_var28',
 'ind_var27',
 'ind_var41',
 'ind_var46_0',
 'ind_var46',
 'num_var27_0',
 'num_var28_0',
 'num_var28',
 'num_var27',
 'num_var41',
 'num_var46_0',
 'num_var46',
 'saldo_var28',
 'saldo_var27',
 'saldo_var41',
 'saldo_var46',
 'imp_amort_var18_hace3',
 'imp_amort_var34_hace3',
 'imp_reemb_var13_hace3',
 'imp_reemb_var33_hace3',
 'imp_trasp_var17_out_hace3',
 'imp_trasp_var33_out_hace3',
 'num_var2_0_ult1',
 'num_var2_ult1',
 'num_reemb_var13_hace3',
 'num_reemb_var33_hace3',
 'num_trasp_var17_out_hace3',
 'num_trasp_var33_out_hace3',
 'saldo_var2_ult1',
 'saldo_medio_var13_medio_hace3']

Uma vez determinada quais as features são constantes, podemos remove-las dos nossos dados utilizando o método transform.

In [7]:
# Removendo as features constantes dos datasets
treino = vt.transform(treino)
teste  = vt.transform(teste)

In [8]:
# Shapes dos datasets
print('Dados de treino:', treino.shape)
print('Dados de teste :', teste.shape)

Dados de treino: (76020, 335)
Dados de teste : (75818, 335)


Pronto! Os dadasets de treino e de teste estão sem as features constantes. Realmente é muito simples fazer e remoção e podemos notar a grande eficácia que o método traz para a análise, reduzindo o dataset consideravelmente.

Agora vamos focar nossa análise nas **features quase-constantes**. A metodologia empregada é a mesma que foi desenvolvida anteriormente, com uma única diferença. Dessa vez, o threshold terá o valor igual a 0,01, ou seja, queremos remover as variáveis que apresentarem inferior a este limite. Portanto, irei apenas apresentar o código, uma vez que a explicação é a mesma.

In [9]:
# Carregando novamente os dados de treino e teste
treino = pd.read_csv('C:/Users/otavio/Mundo/medium/data/santander-train.csv', index_col = 0)
teste  = pd.read_csv('C:/Users/otavio/Mundo/medium/data/santander-test.csv', index_col = 0)

In [10]:
# Removendo coluna TARGET dos dados de treino
treino.drop(labels = ['TARGET'], axis = 1, inplace = True)

In [11]:
# Utilizando sklearn VarianceThreshold para encontrar as features quase-constantes
vt = VarianceThreshold(threshold = 0.01) # 
vt.fit(treino) # fit encontra as features com baixa variância

VarianceThreshold(threshold=0.01)

In [12]:
# get_support é um vetor booleano que indica quais features são retidas
# se somarmos get_support, obtemos quantas features NÃO são quase-constantes
sum(vt.get_support())

272

In [13]:
# Features quase-constantes
features_quaseConstantes = [feature for feature in treino.columns if feature not in treino.columns[vt.get_support()]]
print('Número de features quase-constantes: ', len(features_quaseConstantes))
features_quaseConstantes

Número de features quase-constantes:  97


['ind_var1',
 'ind_var2_0',
 'ind_var2',
 'ind_var6_0',
 'ind_var6',
 'ind_var13_largo',
 'ind_var13_medio_0',
 'ind_var13_medio',
 'ind_var14',
 'ind_var17_0',
 'ind_var17',
 'ind_var18_0',
 'ind_var18',
 'ind_var19',
 'ind_var20_0',
 'ind_var20',
 'ind_var27_0',
 'ind_var28_0',
 'ind_var28',
 'ind_var27',
 'ind_var29_0',
 'ind_var29',
 'ind_var30_0',
 'ind_var31_0',
 'ind_var31',
 'ind_var32_cte',
 'ind_var32_0',
 'ind_var32',
 'ind_var33_0',
 'ind_var33',
 'ind_var34_0',
 'ind_var34',
 'ind_var40',
 'ind_var41',
 'ind_var39',
 'ind_var44_0',
 'ind_var44',
 'ind_var46_0',
 'ind_var46',
 'num_var6_0',
 'num_var6',
 'num_var13_medio_0',
 'num_var13_medio',
 'num_var18_0',
 'num_var18',
 'num_var27_0',
 'num_var28_0',
 'num_var28',
 'num_var27',
 'num_var29_0',
 'num_var29',
 'num_var33',
 'num_var34_0',
 'num_var34',
 'num_var41',
 'num_var46_0',
 'num_var46',
 'saldo_var28',
 'saldo_var27',
 'saldo_var41',
 'saldo_var46',
 'imp_amort_var18_hace3',
 'imp_amort_var34_hace3',
 'imp_reemb

Como resultado obtemos que 97 features podem ser consideradas como quase-constantes. Lembrando que os datasets possuem 369 features.

In [14]:
# Porcentagem de observação mostrando cada um dos diferentes valores
treino['ind_var1'].value_counts() / np.float(len(treino)) * 100

0    99.623783
1     0.376217
Name: ind_var1, dtype: float64

In [15]:
# Removendo as features quase-constantes dos datasets
treino = vt.transform(treino)
teste  = vt.transform(teste)

In [16]:
# Shapes dos datasets
print('Dados de treino:', treino.shape)
print('Dados de teste :', teste.shape)

Dados de treino: (76020, 272)
Dados de teste : (75818, 272)


Aqui está o resultado final. Os datasets de treino e teste foram reduzidos para 272 features. 

Portanto, foi possível demonstrar como esse método simples pode nos auxiliar para fazer a avaliação e remoção de features constantes e quase-constantes, principalmente quando o dataset de estudo possui um número elevado de variáveis. O dataset teve a exclusão de 97 features que não seriam úteis para o desenvolvimento do algoritmo de machine learning. 

A documentação oficial pode ser encontrada aqui https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html.

Outras referências legais sobre o assunto:
- https://www.kaggle.com/prashant111/comprehensive-guide-on-feature-selection
- https://trainindata.medium.com/feature-selection-for-machine-learning-a-comprehensive-overview-bd571db5dd2d
- https://towardsdatascience.com/feature-selection-in-python-using-filter-method-7ae5cbc4ee05

Eu sou o Otavio, estusiasta de dados, ciência e tecnologia. Se você tem alguma crítica, dúvida ou sugestão pode entrar em contato comigo através do e-mail otaviosanluz@gmail.com.

## Obrigado!